# MegaDescriptor-L-384
- Run inference with MegaDescriptor-L-384 (https://huggingface.co/BVRA/MegaDescriptor-L-384)

In [1]:
import pandas as pd
from torchvision import transforms as T
from timm import create_model

from wildlife_tools.features import DeepFeatures
from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier


datasets = [
    'BirdIndividualID',
    'SealID',
    'FriesianCattle2015',
    'ATRW',
    'NDD20',
    'SMALST',
    'SeaTurtleIDHeads',
    'AAUZebraFish',
    'CZoo',
    'CTai',
    'Giraffes',
    'HyenaID2022',
    'MacaqueFaces',
    'OpenCows2020',
    'StripeSpotter',
    'AerialCattle2017',
    'GiraffeZebraID',
    'IPanda50',
    'WhaleSharkID',
    'FriesianCattle2017',
    'Cows2021',
    'LeopardID2022',
    'NOAARightWhale',
    'HappyWhale',
    'HumpbackWhaleID',
    'LionData',
    'NyalaData',
    'ZindiTurtleRecall',
    'BelugaID',
    ]

model = create_model("hf-hub:BVRA/wildlife-mega-L-384", pretrained=True)
extractor = DeepFeatures(model, device='cuda')

root_images = '../data/images/size-518'
root_metadata = '../data/metadata/datasets'

/mnt/appl/software/PyTorch/2.0.1-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /tmp/eb-build/PyTorch/2.0.1/foss-2022a-CUDA-11.7.0/pytorch-v2.0.1/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
results = {}
for name in datasets:
    metadata = pd.read_csv(f'{root_metadata}/{name}/metadata.csv', index_col=0)

    transform = T.Compose([
        T.Resize(size=(384, 384)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

    database = WildlifeDataset(
        metadata=metadata,
        root=f'{root_images}/{name}/',
        transform=transform,
        split=SplitMetadata('split', 'train'),
    )

    query = WildlifeDataset(
        metadata=metadata,
        root=f'{root_images}/{name}/',
        transform=transform,
        split=SplitMetadata('split', 'test'),
    )

    matcher = CosineSimilarity()
    similarity = matcher(query=extractor(query), database=extractor(database))
    preds = KnnClassifier(k=1, database_labels=database.labels_string)(similarity['cosine'])

    acc = sum(preds == query.labels_string) / len(preds)
    print(name, acc)
    results[name] = acc


pd.Series(results).to_csv('results/MegaDescriptor-L-384.csv')

100%|█████████████████████████████████████████████████████████████| 327/327 [19:57<00:00,  3.66s/it]


BirdIndividualID 0.978194338179036


100%|███████████████████████████████████████████████████████████████| 13/13 [00:49<00:00,  3.79s/it]


SealID 0.7865707434052758


100%|█████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.57s/it]


...

NyalaData 0.3644859813084112


100%|███████████████████████████████████████████████████████████████| 76/76 [04:30<00:00,  3.56s/it]


ZindiTurtleRecall 0.7439974042829332


100%|███████████████████████████████████████████████████████████████| 37/37 [03:05<00:00,  5.02s/it]


BelugaID 0.6648394675019577
